In [ ]:
import xarray as xr
from glob import glob
from os import path
import os
import numpy as np

# create NOAA OISST zarr data files



In [ ]:
adir = 'F:/data/sat_data/sst/noaa_oisst/www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/'
dir_pattern_zarr = adir + 'avhrr_zarr/'
dir_pattern = adir + 'avhrr/'
pattern = dir_pattern + '/*/*.nc'
files = glob(pattern)
print('number of files:',len(files))

In [ ]:
%%time
#open dataset, this will take a while
ds=xr.open_mfdataset(files,combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds.isel(zlev=0)
ds_all

In [ ]:
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
ds_all = ds_all.chunk({'time':1000,'lat':300,'lon':300})
ds_all

In [ ]:
#output data to zarr format
ds_all.to_zarr(dir_pattern_zarr, consolidated=True)

In [ ]:
adir = 'F:/data/sat_data/sst/noaa_oisst/www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/'
dir_pattern_zarr = adir + 'avhrr_zarr/'
ds = xr.open_zarr(dir_pattern_zarr, consolidated=True)
ds

In [ ]:
ds.sst[0,:,:].plot()